In [2]:
%pip install ollama
%pip install langchain chromadb gradio 
%pip install -U langchain-community
%pip install pymupdf


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import necessary packages
import ollama  
import gradio as gr  # frontend

# Document processing and retrieval  
from langchain_community.document_loaders import PyMuPDFLoader  
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import Chroma  

# Embedding generation  
from langchain_community.embeddings import OllamaEmbeddings  

import re 

/Users/nekonyo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
response = ollama.chat(
    model="deepseek-r1:1.5b",  
    messages=[
        {"role": "user", "content": "who is naruto"}, 
    ],
)

In [5]:
print(response["message"]["content"])

<think>

</think>

Naruto is a popular Japanese manga, graphic novel, and anime that follows the life of Naruto Uzumaki, a young ninja who becomes the head of the fictional ninja organization Naruto Inc. The manga was first published in 2004 and has since become one of the most widely read and beloved anime in Japan and around the world.

Naruto is set during the Edo period (1603–1868) in Northern Japan, a time of historically difficult relationships between the ruling shogunate and their subjects. The story follows Naruto as he navigates the social and political challenges of that era, while also exploring themes of personal growth, friendship, and the importance of honor.

Key characters include Naruto Uzumaki, his friends, and various opponents such as Sasuke Uchiha, Sakura Haruno, and Jujutsu Kaisen. The manga is known for its complex characters, rich world-building, and deep themes of respect, honor, and the balance between individuality and collective responsibility.

Naruto has 

In [6]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(data)
    embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b")
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="./chroma_db")
    retriever = vectorstore.as_retriever()
    
    return splitter, vectorstore, retriever

In [7]:
def combine_docs(docs):
    return '\n\n'.join(docs.page_content for docs in docs)

In [8]:
def ollama_llm(question, context):
    
    formattted_prompt= f"Question: {question} \n\n Context: {context}"
    response = ollama.chat(
        model="deepseek-r1:1.5b",  
        messages=[
            {"role": "user", "content": formattted_prompt}, 
        ],
    )
    response_context = response["message"]["content"]
    final_answer = re.sub(r'<think>.*?</think>', '', response_context, flags=re.DOTALL).strip() 
    
    return final_answer

In [9]:
#RAP pipeline
def rag_chain(question, splitter, vectorstore, retriever):
    
    retriever_docs = retriever.invoke(question)
    formatted_content = combine_docs(retriever_docs)
    return ollama_llm(question, formatted_content)

In [10]:
def ask_question(pdf_bytes, question):
    splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    if splitter is None:
        return None 
    result = rag_chain(question, splitter, vectorstore, retriever)
    return result

In [11]:
import gradio as gr

# Store uploaded PDFs globally
uploaded_pdfs = []

# Function to add uploaded PDFs
def add_pdfs(new_files):
    global uploaded_pdfs
    if new_files:
        uploaded_pdfs.extend(new_files)  # Add new files to existing ones
        return f"Uploaded {len(uploaded_pdfs)} files."
    return "No new files uploaded."

# Function to process and answer a question using all PDFs
def ask_question(question):
    global uploaded_pdfs
    if not uploaded_pdfs:
        return "No PDFs uploaded."

    responses = []
    for pdf_file in uploaded_pdfs:
        pdf_bytes = pdf_file.read()  # Read each PDF file
        response = process_pdf_and_answer(pdf_bytes, question)  # Call processing function
        responses.append(response)

    return "\n\n".join(responses)

# Dummy function simulating RAG processing
def process_pdf_and_answer(pdf_bytes, question):
    # Placeholder for real PDF processing & RAG model call
    return f"Answer to '{question}' based on one of the PDFs."

# Function to reset everything
def reset():
    global uploaded_pdfs
    uploaded_pdfs = []  # Clear uploaded PDFs
    return "", "", ""  # Clear file status, question input, and output

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# Multi-PDF Upload & Resubmit Questions")

    with gr.Row():
        file_upload = gr.Files(label="Upload PDFs")  
        upload_button = gr.Button("Add PDFs")

    file_status = gr.Textbox(label="Uploaded Files", interactive=False)

    with gr.Row():
        question_input = gr.Textbox(label="Enter your question")
        ask_button = gr.Button("Ask")

    output = gr.Textbox(label="Answer", interactive=False)

    with gr.Row():
        reset_button = gr.Button("Restart")  # Reset button

    # Add PDFs to global list
    upload_button.click(add_pdfs, inputs=[file_upload], outputs=[file_status])

    # Ask questions using uploaded PDFs
    ask_button.click(ask_question, inputs=[question_input], outputs=[output])

    # Reset everything when restart button is clicked
    reset_button.click(reset, inputs=[], outputs=[file_status, question_input, output])

# Launch the Gradio app
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
